In [0]:
%python
%pip install azure-eventhub

In [0]:
dbutils.library.restartPython() 

In [0]:
from azure.eventhub import EventHubProducerClient, EventData
import requests
import json

import os

# OpenWeatherMap API Config
# Obtain Azure Key Vault access key

WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")
CITY = "Porto"
URL = f"http://api.openweathermap.org/data/2.5/weather?q={CITY}&appid={WEATHER_API_KEY}"

# Event Hubs Config
CONNECTION_STRING = os.getenv("CONNECTION_STRING")
EVENT_HUB_NAME =  "weather-data"

# Capture data from OpenWeatherMap API
response = requests.get(URL)
weather_data = response.json()

print("Dados meteorológicos capturados:", weather_data)

# Create a Event Hub Producer Client
producer = EventHubProducerClient.from_connection_string(
    conn_str=CONNECTION_STRING,
    eventhub_name=EVENT_HUB_NAME
)

# Send a single mensage to all the subscribers
try:
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json.dumps(weather_data)))
    producer.send_batch(event_data_batch)
    print("Mensagem enviada com sucesso ao Event Hub!")
finally:
    producer.close()
